# Creating Hundred-Level Dataset

#### Imports

In [1]:
import os
import ast
import json
import shutil
import platform
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()
if platform.node() == 'Nick_Laptop':
    drive = 'C'
elif platform.node() == 'MSI':
    drive = 'D'
else:
    drive = 'uhhhhhh'
    print('Uhhhhhhhhhhhhh')
os.chdir(f'{drive}:/PhD/DissolutionProgramming/LND---Land-Paper')

PROCESSED = 'Data/Processed'
SURNAMES = f'{PROCESSED}/surname_info'
RAW = 'Data/Raw'


#### Loading Data and Dictionaries

In [2]:
with open(f'{PROCESSED}/parish_correction.json', 'r') as j:
    parish_name_correction = json.loads(j.read())
with open(f'{PROCESSED}/hundred_correction.json', 'r') as j:
    hundred_name_correction = json.loads(j.read())
with open(f'{PROCESSED}/parishes_to_hundreds.json', 'r') as j:
    ph_dict = json.loads(j.read())
with open(f'{PROCESSED}/hundreds_damage.json', 'r') as j:
    hundreds_damage = json.loads(j.read())

with open(f'{SURNAMES}/treated_id_dict.json', 'r') as j:
    treated_id_dict = json.loads(j.read())
with open(f'{SURNAMES}/control_id_dict.json', 'r') as j:
    control_id_dict = json.loads(j.read())
    
parish_df = pd.read_csv(f'{PROCESSED}/devon_parish_flows.csv')
parish_df = parish_df.rename(columns={'PAR': 'parish'})
parish_df['hundred'] = parish_df['hundred'].str.title()
parish_df['parish'] = parish_df['parish'].str.title()
parish_df['parish'] = parish_df['parish'].replace(parish_name_correction)


censusdf = pd.read_csv(f'{RAW}/devon_hundreds_census.csv', encoding='utf-8')


mdf = pd.read_csv(f'{PROCESSED}/master_subsidy_data_final_with_parish_info.csv',
                 encoding='utf-8')
mdf['count'] = 1
mdf['max_val'] = mdf['value']

tdf = pd.read_csv(f'{PROCESSED}/tithe_landowners_final.csv',
                 encoding='utf-8')
tdf.rename(columns={'area_perches': 'value'}, inplace=True)
tdf['count'] = 1
tdf['max_val'] = tdf['value']
tdf['parish'] = tdf['parish'].str.title()
tdf['parish'] = tdf['parish'].replace(parish_name_correction)
tdf['hundred'] = tdf['parish'].map(ph_dict)
tdf['hundred'] = tdf['hundred'].str.title()

C:\Users\nicho\AppData\Local\Temp\ipykernel_20472\584097898.py:25: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  mdf = pd.read_csv(f'{PROCESSED}/master_subsidy_data_final_with_parish_info.csv',


#### Aggregating Master Subsidy Data by Hundred

In [4]:
id_vars = ['unique_id', 'group_id', 'combined_id', 'metaphone_id', 'master_id']
agg_dict = {
    'value': 'sum',
    'count': 'sum',
    'max_val': 'max',
    'surname': 'first',    }
missing_hundreds = []
oligarchy_dict = {}
for id_var in id_vars:
    id_oligarchy_dict = {}
    id_type = id_var.split('_')[0]
    new_agg_dict = agg_dict.copy()
    new_agg_dict[f'{id_type}_treatment'] = 'mean'
    new_agg_dict[f'{id_type}_control'] = 'mean'
    
    gdf = mdf.groupby(['hundred', 'year', id_var]).agg(new_agg_dict)
    gdf.rename(columns={'value': 'tot_val'}, inplace=True)
    gdf['avg_val'] = gdf['tot_val'] / gdf['count']


    # Get share of wealth held by top family, top 10% of families in each hundred and year
    for hundred in gdf.index.get_level_values('hundred').unique():
        for year in gdf.index.get_level_values('year').unique():
            hundred_year_df = gdf.xs((hundred, year), level=('hundred', 'year'))
            hundred_year_df['tot_val'] = hundred_year_df['tot_val'].astype(float)
            if len(hundred_year_df) == 0:
                continue
            top_fam = hundred_year_df['tot_val'].max() / hundred_year_df['tot_val'].sum()
            top_10pct = hundred_year_df['tot_val'].nlargest(int(len(hundred_year_df) * 0.1)).sum() / hundred_year_df['tot_val'].sum()
            id_oligarchy_dict[(hundred, year)] = {
                'top_fam': top_fam,
                'top_10pct': top_10pct
                
            }

    reshaped = gdf.unstack('year')
    reshaped.columns = [f'{col}_{year}' for col, year in reshaped.columns]
    reshaped.reset_index(inplace=True)
    
    gtdf = tdf.groupby(['hundred', id_var]).agg(new_agg_dict)
    gtdf.rename(columns={'value': 'tot_val'}, inplace=True)
    gtdf['avg_val'] = gtdf['tot_val'] / gtdf['count']
    gtdf.rename(columns={col: f'{col}_1840' for col in gtdf.columns if 'id' not in col and 'hundred' not in col}, inplace=True)
    gtdf.reset_index(inplace=True)
    for hundred in gtdf['hundred'].unique():
        hundred_df = gtdf[gtdf['hundred'] == hundred]
        if len(hundred_df) == 0:
            continue
        top_fam = hundred_df['tot_val_1840'].max() / hundred_df['tot_val_1840'].sum()
        top_10pct = hundred_df['tot_val_1840'].nlargest(int(len(hundred_df) * 0.1)).sum() / hundred_df['tot_val_1840'].sum()
        id_oligarchy_dict[(hundred, 1840)] = {
            'top_fam': top_fam,
            'top_10pct': top_10pct
        }
    oligarchy_dict[id_var] = id_oligarchy_dict

    merged = pd.merge(reshaped, gtdf, on=['hundred', id_var], how='outer')

    merged.to_csv(f'{SURNAMES}/hundreds_{id_var}_subsidy_data.csv', index=False, encoding='utf-8')

C:\Users\nicho\AppData\Local\Temp\ipykernel_20472\3761891075.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hundred_year_df['tot_val'] = hundred_year_df['tot_val'].astype(float)
C:\Users\nicho\AppData\Local\Temp\ipykernel_20472\3761891075.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hundred_year_df['tot_val'] = hundred_year_df['tot_val'].astype(float)
C:\Users\nicho\AppData\Local\Temp\ipykernel_20472\3761891075.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

#### Collapse the Hundred-Surname Data Into Hundred Data

In [5]:

agg_dict = {
    'value': 'sum',
    'count': 'sum',
    'max_val': 'max',
    
}

for id_var in id_vars:
    id_type = id_var.split('_')[0]
    agg_dict[f'{id_type}_treatment'] = 'mean'
    agg_dict[f'{id_type}_control'] = 'mean'
    mdf[f'{id_type}_treatment_value'] = mdf[f'{id_type}_treatment']
    tdf[f'{id_type}_treatment_value'] = tdf[f'{id_type}_treatment']
    agg_dict[f'{id_type}_treatment_value'] = 'sum'
    mdf[f'{id_type}_control_value'] = mdf[f'{id_type}_control']
    tdf[f'{id_type}_control_value'] = tdf[f'{id_type}_control']
    agg_dict[f'{id_type}_control_value'] = 'sum'

gdf = mdf.groupby(['hundred', 'year']).agg(agg_dict)
gdf['avg_val'] = gdf['value'] / gdf['count']
reshaped = gdf.unstack('year')
reshaped.columns = [f'{col}_{year}' for col, year in reshaped.columns]
reshaped.columns = [f'hundred_{x.replace('value_', 'val_')}' for x in reshaped.columns]
reshaped.reset_index(inplace=True)
gtdf = tdf.groupby(['hundred']).agg(agg_dict)
gtdf['avg_val'] = gtdf['value'] / gtdf['count']
gtdf.columns = [f'hundred_{x}_1840' for x in gtdf.columns]
merged = pd.merge(reshaped, gtdf, on='hundred', how='outer')

# Add in oligarchy data
oligarchy_df = pd.DataFrame.from_dict(id_oligarchy_dict, orient='index')
oligarchy_df.reset_index(inplace=True)
oligarchy_df.columns = ['hundred', 'year', 'top_fam', 'top_10pct']
# Pivot to top_fam_year and top_10pct_year
oligarchy_df['year'] = oligarchy_df['year'].astype(str)
oligarchy_df = oligarchy_df.pivot(index='hundred', columns='year', values=['top_fam', 'top_10pct'])
oligarchy_df.columns = [f'{col[0]}_{col[1]}' for col in oligarchy_df.columns]


merged = pd.merge(merged, oligarchy_df, on='hundred', how='left')
merged

,hundred,hundred_val_1524,hundred_val_1543,hundred_val_1581,hundred_val_1674,hundred_count_1524,hundred_count_1543,hundred_count_1581,hundred_count_1674,hundred_max_val_1524,...,top_fam_1524,top_fam_1543,top_fam_1581,top_fam_1674,top_fam_1840,top_10pct_1524,top_10pct_1543,top_10pct_1581,top_10pct_1674,top_10pct_1840
0,Axminster,3524.753333,5041.166667,1271.000000,1515.000000,719.0,740.0,318.0,737.0,100.000000,...,0.030640,0.037690,0.027537,0.028383,0.067704,0.436012,0.423414,0.309205,0.372277,0.697989
1,Bampton,1903.166667,2345.000000,691.000000,1376.333333,460.0,579.0,172.0,634.0,80.000000,...,0.042035,0.051173,0.052098,0.018164,0.076602,0.479902,0.449893,0.337192,0.359167,0.623673
2,Black Torrington,5311.996667,7192.666667,2583.000000,2011.500000,1470.0,1608.0,615.0,944.0,133.330000,...,0.025100,0.018074,0.019744,0.014914,0.079705,0.430754,0.418853,0.306620,0.358936,0.665092
3,Braunton,5364.996667,6688.666667,2441.000000,2889.000000,1093.0,1341.0,443.0,1309.0,200.000000,...,0.038956,0.029901,0.032773,0.032537,0.105850,0.495093,0.459534,0.337157,0.404408,0.735877
4,Cliston,1075.023333,1309.000000,533.000000,103.333333,311.0,266.0,149.0,47.0,32.000000,...,0.063254,0.043545,0.050657,0.167742,0.408950,0.421910,0.380443,0.287054,0.293548,0.686395
5,Coleridge,8988.463333,8527.000000,2889.500000,5214.000000,1518.0,1971.0,626.0,2354.0,470.000000,...,0.054106,0.026387,0.020073,0.018987,0.054352,0.590145,0.490051,0.331545,0.386971,0.600478
6,Colyton,2975.563333,2599.083333,871.000000,1272.826667,504.0,379.0,233.0,629.0,66.666667,...,0.048170,0.039245,0.041332,0.022784,0.219192,0.498057,0.378852,0.340987,0.410134,0.808364
7,Crediton,3682.433333,5006.500000,1025.333333,2259.000000,702.0,900.0,316.0,1209.0,100.000000,...,0.032587,0.079896,0.032185,0.028774,0.129718,0.459655,0.489264,0.338427,0.436034,0.663088
8,East Budleigh,5063.800000,4927.333333,1323.000000,NaN,938.0,978.0,401.0,NaN,333.333333,...,0.071685,0.028616,0.065004,NaN,0.336833,0.532168,0.488669,0.380952,NaN,0.841155
9,Ermington,5910.166667,5377.166667,1612.000000,2553.666667,1081.0,959.0,367.0,1159.0,333.000000,...,0.090917,0.055791,0.039702,0.021538,0.101576,0.552269,0.504014,0.380893,0.391724,0.659001


#### Add the Parish-Level Data, Summing or Averaging Appropriately

In [6]:
censusdf['hundred'] = censusdf['hundred'].str.title()
hundred_df = pd.merge(merged, censusdf, on='hundred', how='left')

parish_agg_dict = {'mean_elev': 'mean',
                   'mean_slope': 'mean',
                   'wheatsuit': 'mean',
                   'lspc1332': 'mean',
                   'ind_1831': 'mean',
                   'agr_1831': 'mean',
                   'agr_share': 'mean',
                   'ind_share': 'mean',
                   'mills_1400': 'sum',
                   'gent_1400': 'sum',
                   'NrPatents': 'sum',
                   'copys_1850': 'sum',
                   'mills': 'sum',
                   'NrGentry': 'sum',
                   'area': 'sum',
                   'landOwned': 'sum',
                   'WheatYield': 'mean',
                   'copys_1516': 'sum',
                   'hrv_land': 'sum',
                   'lspc1525': 'mean',
                   'distriver': 'mean',
                   'distmkt': 'mean',
                   'distcoal': 'mean',
                   'latitude': 'mean',
                   'longitude': 'mean',
                   }

pgdf = parish_df.groupby('hundred').agg(parish_agg_dict)
pgdf.reset_index(inplace=True)
pgdf['hundred'] = hundred_df['hundred'].replace(hundred_name_correction)

hundred_df = pd.merge(hundred_df, pgdf, on='hundred', how='left')

hundred_df.to_csv(f'{PROCESSED}/hundred_dataset.csv', index=False, encoding='utf-8')